In [1]:
import pandas as pd



# US

In [27]:
us = pd.read_csv("USvideos.csv")
us.head()
us.dtypes
us['trending_date'] = pd.to_datetime(us['trending_date'], format = '%y.%d.%m')
us['trending_YearMonth'] = us['trending_date'].apply(lambda x: f'{x.year}-{x.month}')
us['publish_time'] = pd.to_datetime(us['publish_time'], format = '%Y-%m-%dT%H:%M:%S.%fZ')
us_max_byMonth = us.groupby(['trending_YearMonth','title'])['views','likes','dislikes'].max()
us_view_max_byMonth = us_max_byMonth['views'].groupby(level=0, group_keys=False)
us_top20_byMonth = us_view_max_byMonth.nlargest(20)

# GB

In [28]:
gb = pd.read_csv("GBvideos.csv")
gb.head()
gb.dtypes
gb['trending_date'] = pd.to_datetime(gb['trending_date'], format = '%y.%d.%m')
gb['trending_YearMonth'] = gb['trending_date'].apply(lambda x: f'{x.year}-{x.month}')
gb['publish_time'] = pd.to_datetime(gb['publish_time'], format = '%Y-%m-%dT%H:%M:%S.%fZ')
gb_max_byMonth = gb.groupby(['trending_YearMonth','title'])['views','likes','dislikes'].max()
gb_view_max_byMonth = gb_max_byMonth['views'].groupby(level=0, group_keys=False)
gb_top20_byMonth = gb_view_max_byMonth.nlargest(20)

# CA

In [29]:
ca = pd.read_csv("GBvideos.csv")
ca.head()
ca.dtypes
ca['trending_date'] = pd.to_datetime(ca['trending_date'], format = '%y.%d.%m')
ca['trending_YearMonth'] = ca['trending_date'].apply(lambda x: f'{x.year}-{x.month}')
ca['publish_time'] = pd.to_datetime(ca['publish_time'], format = '%Y-%m-%dT%H:%M:%S.%fZ')
ca_max_byMonth = ca.groupby(['trending_YearMonth','title'])['views','likes','dislikes'].max()
ca_view_max_byMonth = ca_max_byMonth['views'].groupby(level=0, group_keys=False)
ca_top20_byMonth = ca_view_max_byMonth.nlargest(20)

# DE

In [31]:
de = pd.read_csv("GBvideos.csv")
de.head()
de.dtypes
de['trending_date'] = pd.to_datetime(de['trending_date'], format = '%y.%d.%m')
de['trending_YearMonth'] = de['trending_date'].apply(lambda x: f'{x.year}-{x.month}')
de['publish_time'] = pd.to_datetime(de['publish_time'], format = '%Y-%m-%dT%H:%M:%S.%fZ')
de_max_byMonth = de.groupby(['trending_YearMonth','title'])['views','likes','dislikes'].max()
de_view_max_byMonth = de_max_byMonth['views'].groupby(level=0, group_keys=False)
de_top20_byMonth = de_view_max_byMonth.nlargest(20)

# FR

In [32]:
fr = pd.read_csv("GBvideos.csv")
fr.head()
fr.dtypes
fr['trending_date'] = pd.to_datetime(fr['trending_date'], format = '%y.%d.%m')
fr['trending_YearMonth'] = fr['trending_date'].apply(lambda x: f'{x.year}-{x.month}')
fr['publish_time'] = pd.to_datetime(fr['publish_time'], format = '%Y-%m-%dT%H:%M:%S.%fZ')
fr_max_byMonth = fr.groupby(['trending_YearMonth','title'])['views','likes','dislikes'].max()
fr_view_max_byMonth = fr_max_byMonth['views'].groupby(level=0, group_keys=False)
fr_top20_byMonth = fr_view_max_byMonth.nlargest(20)